In [38]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, cross
import pandas as pd
import pandas_ta as ta
from binance.client import Client
client = Client()

In [40]:
def getdata(symbol,start):
    frame = pd.DataFrame(client.get_historical_klines(symbol,'4h',start))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time','Open','High','Low','Close','Volume']
    frame.set_index('Time',inplace=True)
    frame.index = pd.to_datetime(frame.index,unit='ms')
    frame = frame.astype(float)
    return frame

In [58]:
df = getdata('ETHUSDT','2021-01-01')


In [109]:
class SmaCross(Strategy):

    n1 = 13
    n2 = 30
    in_position = False

    def init(self):
        close = self.data.Close
        self.ema = self.I(ta.ema , pd.Series(close), self.n1)
        self.sma = self.I(ta.sma, pd.Series(close), self.n2)


    def next(self):
        
        if crossover(self.ema, self.sma):#and self.in_position == False:
            price = self.data.Low[-1]
            two_lows_atras = self.data.Low[-2]
            tpro = crossover(self.sma,self.ema)
            self.buy(size=0.05, sl=price - 1) # ,tp=tpro)
            self.in_position = True
        elif crossover(self.sma,self.ema):# and self.in_position == True:
            #one_high = self.data.High[-1]
            #two_highs_atras = self.data.High[-2]
            #tpro = crossover(self.ema,self.sma)
            self.sell(size=0.05)#,sl=(two_highs_atras-one_high)) #,tp=tpro)
            self.in_position = False
        # elif crossover(self.sma,self.ema) and self.in_position == False:
        #     sloss = self.data.High[-2]
        #     self.sell(size=0.05,sl=sloss)
        #     self.in_position = True
        # elif crossover(self.ema, self.sma) and self.in_position == True:
        #     sloss = self.data.Low[-2]
        #     self.buy(size=0.05,sl=sloss)
        #     self.in_position = False


            
        #elif cross(self.sma, self.ema):
        #    self.sell()

In [110]:
bt = Backtest(df,SmaCross,cash=100000,margin=1/20,commission=0.002,exclusive_orders=True)


In [111]:
output = bt.run()

In [112]:
#output = pd.DataFrame(output)
#output.to_csv('/Users/oscarnicolas/Downloads/a/cross_EMA13_SMA30.csv')
output

Start                     2021-01-01 00:00:00
End                       2022-12-15 16:00:00
Duration                    713 days 16:00:00
Exposure Time [%]                   71.935559
Equity Final [$]                  38628.96814
Equity Peak [$]                  108614.22172
Return [%]                         -61.371032
Buy & Hold Return [%]               71.054751
Return (Ann.) [%]                  -38.506653
Volatility (Ann.) [%]               53.496599
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -74.36717
Avg. Drawdown [%]                   -74.36717
Max. Drawdown Duration      703 days 04:00:00
Avg. Drawdown Duration      703 days 04:00:00
# Trades                                  166
Win Rate [%]                        24.096386
Best Trade [%]                      48.548528
Worst Trade [%]                    -18.990116
Avg. Trade [%]                    

In [113]:
bt.plot()



Row(id='14193', ...)